#### Adicionando os paths

In [5]:
import sys
sys.path.append('../')
sys.path.append('../Scripts/')

#### Importando bibliotecas

In [6]:
import pandas as pd
from Global_Variables import *
import Aux_Functions as af

In [8]:
atributos = pd.read_csv("../Outputs/Tabela_Atributos.csv", sep=";")

In [9]:
atributos

,Sequence,Genotype,Position,Codon,Drug,Codon_Resistente_01,Codon_Resistente_02,Codon_Resistente_03,Codon_Resistente_04,Codon_Resistente_05,...,Codon_Usage_09,Barreira_Genetica_10,Prevalencia_10,Codon_Usage_10,Barreira_Genetica_11,Prevalencia_11,Codon_Usage_11,Barreira_Genetica_12,Prevalencia_12,Codon_Usage_12
0,MXYZPTLK,1b,24,CAG,-,-,-,-,-,-,...,0.00,-1.0,0.0,0.00,-1,0.000000,0.00,-1,0.0,0.00
1,MXYZPTLK,1b,28,CTG,DCV,ATG,-,-,-,-,...,0.00,-1.0,0.0,0.00,-1,0.000000,0.00,-1,0.0,0.00
2,MXYZPTLK,1b,28,CTG,EBR,ATG,-,-,-,-,...,0.00,-1.0,0.0,0.00,-1,0.000000,0.00,-1,0.0,0.00
3,MXYZPTLK,1b,28,CTG,LDV,ATG,-,-,-,-,...,0.00,-1.0,0.0,0.00,-1,0.000000,0.00,-1,0.0,0.00
4,MXYZPTLK,1b,28,CTG,OMV,ATG,-,-,-,-,...,0.00,-1.0,0.0,0.00,-1,0.000000,0.00,-1,0.0,0.00
5,MXYZPTLK,1b,30,CGG,DCV,CAA,CAG,-,-,-,...,0.00,-1.0,0.0,0.00,-1,0.000000,0.00,-1,0.0,0.00
6,MXYZPTLK,1b,30,CGG,EBR,CAA,CAG,-,-,-,...,0.00,-1.0,0.0,0.00,-1,0.000000,0.00,-1,0.0,0.00
7,MXYZPTLK,1b,30,CGG,LDV,CAA,CAG,-,-,-,...,0.00,-1.0,0.0,0.00,-1,0.000000,0.00,-1,0.0,0.00
8,MXYZPTLK,1b,30,CGG,OMV,CAA,CAG,-,-,-,...,0.00,-1.0,0.0,0.00,-1,0.000000,0.00,-1,0.0,0.00
9,MXYZPTLK,1b,31,TTG,DCV,ATG,TTC,TTT,GTA,GTC,...,0.00,-1.0,0.0,0.00,-1,0.000000,0.00,-1,0.0,0.00
